# IR24 Project Testing Notebook

#### Import Libraries

In [1]:
from llama_index.core import SummaryIndex
from llama_index.readers.web import SimpleWebPageReader
from IPython.display import Markdown, display
import IR24_data

## Testing Data
### Wikipedia
https://en.wikipedia.org/wiki/History_of_Singapore

https://en.wikipedia.org/wiki/Early_history_of_Singapore 

https://en.wikipedia.org/wiki/Kingdom_of_Singapura 

https://en.wikipedia.org/wiki/Founding_years_of_modern_Singapore 

https://en.wikipedia.org/wiki/Singapore_in_the_Straits_Settlements 

https://en.wikipedia.org/wiki/Fall_of_Singapore

https://en.wikipedia.org/wiki/Japanese_occupation_of_Singapore 

https://en.wikipedia.org/wiki/Operation_Tiderace 

https://en.wikipedia.org/wiki/Colony_of_Singapore 

https://en.wikipedia.org/wiki/Self-governance_of_Singapore 

https://en.wikipedia.org/wiki/Singapore_in_Malaysia 

https://en.wikipedia.org/wiki/History_of_the_Republic_of_Singapore 

## Create docs

In [2]:
#urls = ["https://en.wikipedia.org/wiki/History_of_Singapore","https://en.wikipedia.org/wiki/Early_history_of_Singapore","https://en.wikipedia.org/wiki/Kingdom_of_Singapura","https://en.wikipedia.org/wiki/Founding_years_of_modern_Singapore","https://en.wikipedia.org/wiki/Singapore_in_the_Straits_Settlements","https://en.wikipedia.org/wiki/Fall_of_Singapore","https://en.wikipedia.org/wiki/Japanese_occupation_of_Singapore","https://en.wikipedia.org/wiki/Operation_Tiderace","https://en.wikipedia.org/wiki/Colony_of_Singapore","https://en.wikipedia.org/wiki/Self-governance_of_Singapore","https://en.wikipedia.org/wiki/Singapore_in_Malaysia", "https://en.wikipedia.org/wiki/History_of_the_Republic_of_Singapore"] 
articles = ["History_of_Singapore",
            "Early_history_of_Singapore",
            "Kingdom_of_Singapura",
            "Founding_years_of_modern_Singapore",
            "Singapore_in_the_Straits_Settlements",
            "Fall_of_Singapore",
            "Japanese_occupation_of_Singapore",
            "Operation_Tiderace",
            "Colony_of_Singapore",
            "Self-governance_of_Singapore",
            "Singapore_in_Malaysia", 
            "History_of_the_Republic_of_Singapore"] 

for article in articles:
    IR24_data.wiki_to_text(article, "wikidata/" + article + ".txt")
    #IR24_data.html_to_markdown(url, "wikidata/"+url[30:]+".md")

History_of_Singapore's content saved successfully!
Early_history_of_Singapore's content saved successfully!
Kingdom_of_Singapura's content saved successfully!
Founding_years_of_modern_Singapore's content saved successfully!
Singapore_in_the_Straits_Settlements's content saved successfully!
Fall_of_Singapore's content saved successfully!
Japanese_occupation_of_Singapore's content saved successfully!
Operation_Tiderace's content saved successfully!
Colony_of_Singapore's content saved successfully!
Self-governance_of_Singapore's content saved successfully!
Singapore_in_Malaysia's content saved successfully!
History_of_the_Republic_of_Singapore's content saved successfully!


In [27]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Split docs into chunks
# https://python.langchain.com/docs/how_to/recursive_text_splitter/
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
    is_separator_regex=False,
)

texts = []

for article in articles:
    path = "wikidata/" + article + ".txt"
    with open(path, encoding='UTF-8') as f:
        article = f.read()
        print("Splitting: ", path)
        doc_texts = text_splitter.create_documents([article])
        texts.append(doc_texts)
    

print(len(texts))
for text in texts:
    print(text)

print(texts[1])

Splitting:  wikidata/History_of_Singapore.txt
Splitting:  wikidata/Early_history_of_Singapore.txt
Splitting:  wikidata/Kingdom_of_Singapura.txt
Splitting:  wikidata/Founding_years_of_modern_Singapore.txt
Splitting:  wikidata/Singapore_in_the_Straits_Settlements.txt
Splitting:  wikidata/Fall_of_Singapore.txt
Splitting:  wikidata/Japanese_occupation_of_Singapore.txt
Splitting:  wikidata/Operation_Tiderace.txt
Splitting:  wikidata/Colony_of_Singapore.txt
Splitting:  wikidata/Self-governance_of_Singapore.txt
Splitting:  wikidata/Singapore_in_Malaysia.txt
Splitting:  wikidata/History_of_the_Republic_of_Singapore.txt
12
[Document(metadata={'start_index': 0}, page_content='The history of the modern state of Singapore dates back to its founding in the\nearly 19th century; however, evidence suggests that a significant trading\nsettlement existed on the island in the 14th century. The last ruler of the\nKingdom of Singapura, Parameswara, was expelled by the Majapahit or the'), Document(metadata=

In [ ]:
# Adapted from: https://docs.llamaindex.ai/en/stable/examples/data_connectors/WebPageDemo/
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://en.wikipedia.org/wiki/History_of_Singapore"]
)
print(documents[0])

index = SummaryIndex.from_documents(documents)
print(index)

Doc ID: https://en.wikipedia.org/wiki/History_of_Singapore
Text: Jump to content  Main menu  Main menu  move to sidebar hide
Navigation    * [Main page](/wiki/Main_Page "Visit the main page
\[z\]")   * [Contents](/wiki/Wikipedia:Contents "Guides to browsing
Wikipedia")   * [Current events](/wiki/Portal:Current_events "Articles
related to current events")   * [Random article](/wiki/Special:Random
"Visit a ran...
